# Generate Hypercomplex numbers based on observations of patterns

The inputs are first the dimension n and will output 2^n elements corrspending to hypercomplex multiplipcation table.
Then the code will print mutiplication table of the n dimension hypercomplex number.
Then the code will perform simple calculation of a * b = c where a is the first expression and b is the second and c is the results.
For the preprint behind this code please visit: https://osf.io/byqfw_v3/

In [ ]:
import re
import csv
from functools import lru_cache

def real_sign(i, j):
    """Returns the sign for the priviously quaternion multiplication e_i * e_j. Now the real."""
    table = [
        [1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 0:  # Base case: previously Quaternion table now real
        dim = 1
        return [[(real_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val  # Block 'a'
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val  # Block 'b'
            table[i + prev_dim][j] = new_val  # Block 'c'
            table[i + prev_dim][j + prev_dim] = new_val  # Block 'd'

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 0 and local_row == 0:
                        s = real_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

def multiply_elements(a, b, n):
    """Multiplies two hypercomplex elements 'a' and 'b'."""
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0] * dim
    for i in range(dim):
        ai = a[i]
        if ai == 0:
            continue
        for j in range(dim):
            bj = b[j]
            if bj == 0:
                continue
            coeff = ai * bj
            s, k = table[i][j]
            result[k] += coeff * s
    return result

def format_element(elem):
    """
    Returns a formatted string representation of a hypercomplex element given
    its list of coefficients.

    For example, [1, 2, -3, 0] becomes "1 + 2e1 - 3e2".
    """
    terms = []
    for i, coeff in enumerate(elem):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        else:
            if coeff == 1:
                term = f"e{i}"
            elif coeff == -1:
                term = f"-e{i}"
            else:
                term = f"{coeff}e{i}"
        terms.append(term)
    if not terms:
        return "0"
    formatted = terms[0]
    for term in terms[1:]:
        formatted += " - " + term[1:] if term.startswith("-") else " + " + term
    return formatted
def parse_element(s, dim):
    """Parses a hypercomplex element string into a list of coefficients."""
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    tokens = re.findall(r'[+-][^+-]+', s)
    coeffs = [0.0] * dim
    for token in tokens:
        if 'e' in token:
            coeff_str, index_str = token.split('e', 1)
            #coeff = 1.0 if coeff_str in ("+", "-") else float(coeff_str)
            if(coeff_str in ("+")):
                coeff=1.0
            elif (coeff_str in ("-")):
                coeff=-1.0
            else:
                coeff=float(coeff_str)
            index = int(index_str)
        else:
            coeff = float(token)
            index = 0
        if index < 0 or index >= dim:
            raise ValueError(f"Index {index} out of range")
        coeffs[index] += coeff
    return coeffs

def save_table_to_csv(table, filename):
    """Saves the Cayley-Dickson multiplication table to a CSV file."""
    dim = len(table)
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header row (e0, e1, e2, ...)
        header = [f"e{j}" for j in range(dim)]
        writer.writerow([""] + header) # Empty cell in top-left corner

        # Write table data
        for i in range(dim):
            row = [f"e{i}"]  # First element in the row is the basis element
            for sign, index in table[i]:
                row.append(f"{'+' if sign == 1 else '-' }e{index}")
            writer.writerow(row)

def main():
    print("Hypercomplex Element Multiplication & Cayley-Dickson Table Generator")

    try:
        n = int(input("Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): "))
    except ValueError:
        print("Error: n must be an integer.")
        return

    dim = 1 << n
    print(f"\nThis algebra has dimension {dim} (basis elements e0, e1, ..., e{dim-1}).\n")

    # Generate and save the table
    table = cayley_dickson_table(n)
    filename = f"cayley_dickson_table_n{n}.csv"
    save_table_to_csv(table, filename)
    print(f"Cayley-Dickson multiplication table saved to '{filename}'")
    print(f"\nCayley–Dickson Multiplication Table (n={n}, dimension={dim}):")
    for row in table:
        print("\t".join(f"{'' if s == 1 else '-'}e{k}" for s, k in row))
    print("-" * 40)
    # Optional: Perform multiplication (kept from original script)
    while True:
        try:
            a_str = input("Enter the first element (e.g., '1 + 2e1 - 3e2', or 'q' to quit):\n")
            if a_str.lower() == 'q':
                break
            a = parse_element(a_str, dim)

            b_str = input("\nEnter the second element (e.g., '4 - e1 + 0.5e3'):\n")
            b = parse_element(b_str, dim)

            prod = multiply_elements(a, b, n)
            print("\nResult of multiplication:")
            print(f"({format_element(a)}) * ({format_element(b)}) = {format_element(prod)}")

        except ValueError as e:
            print("Error:", e)

if __name__ == "__main__":
    main()

Hypercomplex Element Multiplication & Cayley-Dickson Table Generator
Enter n (n=1 for Complex, n=2 for Quaternions, n=3 for Octonions, etc.): 5

This algebra has dimension 32 (basis elements e0, e1, ..., e31).

Cayley-Dickson multiplication table saved to 'cayley_dickson_table_n5.csv'

Cayley–Dickson Multiplication Table (n=5, dimension=32):
e0	e1	e2	e3	e4	e5	e6	e7	e8	e9	e10	e11	e12	e13	e14	e15	e16	e17	e18	e19	e20	e21	e22	e23	e24	e25	e26	e27	e28	e29	e30	e31
e1	-e0	e3	-e2	e5	-e4	-e7	e6	e9	-e8	-e11	e10	-e13	e12	e15	-e14	e17	-e16	-e19	e18	-e21	e20	e23	-e22	-e25	e24	e27	-e26	e29	-e28	-e31	e30
e2	-e3	-e0	e1	e6	e7	-e4	-e5	e10	e11	-e8	-e9	-e14	-e15	e12	e13	e18	e19	-e16	-e17	-e22	-e23	e20	e21	-e26	-e27	e24	e25	e30	e31	-e28	-e29
e3	e2	-e1	-e0	e7	-e6	e5	-e4	e11	-e10	e9	-e8	-e15	e14	-e13	e12	e19	-e18	e17	-e16	-e23	e22	-e21	e20	-e27	e26	-e25	e24	e31	-e30	e29	-e28
e4	-e5	-e6	-e7	-e0	e1	e2	e3	e12	e13	e14	e15	-e8	-e9	-e10	-e11	e20	e21	e22	e23	-e16	-e17	-e18	-e19	-e28	-e29	-e30	-e31	e24	e25	e26	e27
e5

# Colorizing the table
To colorize the table it must have generated csv file with its dimension e.g colorizing dimension 4 need csv file generated from with hypercomplex generator.
The inputs is dimension n and the block size m that correspendt to mxm size.
Colorize the hypercomplex number based on their elements ignoring the signs or colorizing them based on their signs ignoring the elements.
The code take the output csv file from generating hypercomplex numbers and generate HTML files.
The codes below generate random colors but it will keep the same colors when there is similarities either by signs or elements according to the code functionalities.

## Colorize based on elements ignoring the signs


In [ ]:
import csv
import random

def generate_unique_colors(num_colors):
    """Generates a list of unique, visually distinct RGB colors."""
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)

def generate_colored_html_table(csv_filename, html_filename, n,block_dimension):
    """
    Generates an HTML table, coloring 8x8 blocks based on *content*
    (ignoring signs), and without the first row/column.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // block_dimension) * (dim // block_dimension)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
               table {
  width: 100%; /* Ensure the table takes the full width of its container */
  border-collapse: collapse; /* Optional: Collapse borders for a cleaner look */
}

td {
  width: 60px; /* Set a fixed width for each cell */
  height: 40px; /* Set a fixed height for each cell */
  box-sizing: border-box; /* Include padding and border in the element's total width and height */
  text-align: center; /* Optional: Center the text within the cell */
  border: 1px solid #000; /* Optional: Add a border to the cells */
  color:white;
}
</style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // block_dimension) * block_dimension
            j_block = (j // block_dimension) * block_dimension

            # Create a "signature" for the 8x8 block (ignoring signs)
            block_signature = []
            for row_index in range(i_block, min(i_block + block_dimension, dim)):
                for col_index in range(j_block, min(j_block + block_dimension, dim)):
                    block_signature.append(table_data[row_index][col_index].strip().lstrip('+-')) # Remove signs
            block_signature = tuple(sorted(block_signature)) # Sort to make order-independent

            # Get color for this signature
            if block_signature in block_signatures:
                color = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                block_signatures[block_signature] = color
                next_color_index += 1

            # Generate the HTML for the cell
            cell_value = table_data[i][j]
            html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():

    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
        size=int(input("Enter the value of block size "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same_elements_block_{size}.html"
    block_dimension=2**size
    generate_colored_html_table(csv_filename, html_filename, n,block_dimension)


if __name__ == "__main__":
    main()

Enter the value of n (dimension is 2^n): 3
Enter the value of block size 2
HTML table generated and saved to 'cayley_dickson_table_n3_same_elements_block_2.html'


# Colorizing based on the signs


In [ ]:
import csv
import random
import re

"""def generate_unique_colors(num_colors):
    #Generates a list of unique, visually distinct RGB colors.
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)"""

def generate_unique_colors(num_colors):
    """
    Returns a list of 12 static, visually distinct RGB colors.
    Each color is chosen to be dark enough so that white text appears clearly.
    12 colors it all it needs.
    """
    colors = [
        "#1F77B4",  # A medium dark blue
        "#D62728",  # A strong red
        "#2CA02C",  # A medium dark green
        "#9467BD",  # A muted purple
        "#8C564B",  # A dark brown
        "#004E64",  # A deep teal
        "#000075",  # A navy blue
        "#5C4033",  # A chocolate brown
        "#3B3B3B",  # A dark gray
        "#4B0082",  # An indigo
        "#8B0000",  # A dark red
        "#006400",  # A dark green
    ]
    return colors
def generate_colored_html_table(csv_filename, html_filename, n,block_dimension):
    """
    Generates an HTML table, coloring 8x8 blocks based on *exact* content
    (including signs and order), and without the first row/column.
    Replaces block content with a unique number if the block repeats.
    """

    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    num_colors = (dim // block_dimension) * (dim // block_dimension)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0
    block_number = 1  # Start numbering blocks from 1

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse;color:white }
        td { border: 1px solid black; padding: 5px; text-align: center; }
    </style>
</head>
<body>
    <table>
"""

    for i in range(dim):  # Iterate over all rows
        html_string += "        <tr>\n"
        for j in range(dim):  # Iterate over all columns
            # Determine the block indices (i_block, j_block)
            i_block = (i // block_dimension) * block_dimension
            j_block = (j // block_dimension) * block_dimension

            # Create a "signature" for the 8x8 block (INCLUDING signs and order)
            block_signature = []
            for row_index in range(i_block, min(i_block + block_dimension, dim)):
                for col_index in range(j_block, min(j_block + block_dimension, dim)):
                    block_signature.append(re.sub(r'\d', '',table_data[row_index][col_index]))#table_data[row_index][col_index].strip())
            block_signature = tuple(block_signature)

            # Get color and number for this signature
            if block_signature in block_signatures:
                color, number = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                number = block_number
                block_signatures[block_signature] = (color, number) # Store color AND number
                next_color_index += 1
                block_number += 1

            # Generate the HTML for the cell
            if (i, j) == (i_block, j_block):  # Top-left cell of the block
                html_string += f"            <td style='background-color: {color};'>\"{number}\"</td>\n"
            #else:
                #  Empty cell, but with the correct background color
                #html_string += f"            <td style='background-color: {color};'></td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")
def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
        size=int(input("Enter the block size : "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_same_signs_dimension_{size}.html"
    block_dimension=2**size
    generate_colored_html_table(csv_filename, html_filename, n,block_dimension)

if __name__ == "__main__":
    main()

Enter the value of n (dimension is 2^n): 3
Enter the block size : 2
HTML table generated and saved to 'cayley_dickson_table_n3_same_signs_dimension_2.html'


In [ ]:
import csv
import random

"""def generate_unique_colors(num_colors):
    #Generates a list of unique, visually distinct RGB colors.
    colors = set()
    while len(colors) < num_colors:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        hex_color = "#{:02X}{:02X}{:02X}".format(r, g, b)
        colors.add(hex_color)
    return list(colors)"""

def generate_unique_colors(num_colors):
    """
    Returns a list of 12 static, visually distinct RGB colors.
    Each color is chosen to be dark enough so that white text appears clearly.
    """
    colors = [
        "#1F77B4",  # A medium dark blue
        "#D62728",  # A strong red
        "#2CA02C",  # A medium dark green
        "#9467BD",  # A muted purple
        "#8C564B",  # A dark brown
        "#004E64",  # A deep teal
        "#000075",  # A navy blue
        "#5C4033",  # A chocolate brown
        "#3B3B3B",  # A dark gray
        "#4B0082",  # An indigo
        "#8B0000",  # A dark red
        "#006400",  # A dark green
    ]
    return colors

def extract_sign(cell_value):
    """
    Extracts the sign from a cell value.
    If the cell starts with a '-' then it is negative; otherwise, it is positive.
    """
    cell_value = cell_value.strip()
    if cell_value.startswith('-'):
        return '-'
    return '+'

def generate_colored_html_table(csv_filename, html_filename, n, block_dimension):
    """
    Generates an HTML table where blocks are grouped by both the sign and the order
    of signs of their cells. Only blocks having an identical ordered signature will share a color.
    """
    try:
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.reader(csvfile)
            table_data = list(reader)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_filename}' not found.")
        return

    # Remove the first row and first column (typically headers)
    table_data = [row[1:] for row in table_data[1:]]

    if not table_data:
        print("Error: CSV file is empty.")
        return

    # Determine the dimension of the table
    dim = len(table_data)
    if dim == 0 or (dim & (dim - 1)) != 0:
        print("Error: CSV does not represent a valid Cayley-Dickson table.")
        return

    # Calculate the number of blocks
    num_colors = (dim // block_dimension) * (dim // block_dimension)
    if num_colors == 0:
        num_colors = 1
    colors = generate_unique_colors(num_colors)
    block_signatures = {}
    next_color_index = 0

    html_string = """<!DOCTYPE html>
<html>
<head>
    <title>Cayley-Dickson Multiplication Table</title>
    <style>
        table { border-collapse: collapse; }
        td { border: 1px solid black; padding: 5px; text-align: center;color:white }
    </style>
</head>
<body>
    <table>
"""

    # Process each cell in the table and assign colors based on block signatures
    for i in range(dim):
        html_string += "        <tr>\n"
        for j in range(dim):
            # Determine the starting indices of the block that contains the current cell.
            i_block = (i // block_dimension) * block_dimension
            j_block = (j // block_dimension) * block_dimension

            # Build the block signature in the order the cells appear (without sorting)
            block_signature = []
            for row_index in range(i_block, min(i_block + block_dimension, dim)):
                for col_index in range(j_block, min(j_block + block_dimension, dim)):
                    sign = extract_sign(table_data[row_index][col_index])
                    block_signature.append(sign)
            block_signature = tuple(block_signature)

            # Assign a color to each unique block signature based on the *exact order* of signs.
            if block_signature in block_signatures:
                color = block_signatures[block_signature]
            else:
                color = colors[next_color_index % len(colors)]
                block_signatures[block_signature] = color
                next_color_index += 1

            # Generate the HTML cell with the given color.
            cell_value = table_data[i][j]
            html_string += f"            <td style='background-color: {color};'>{cell_value}</td>\n"
        html_string += "        </tr>\n"

    html_string += """    </table>
</body>
</html>
"""

    try:
        with open(html_filename, 'w') as htmlfile:
            htmlfile.write(html_string)
        print(f"HTML table generated and saved to '{html_filename}'")
    except Exception as e:
        print(f"Error writing HTML file: {e}")

def main():
    try:
        n = int(input("Enter the value of n (dimension is 2^n): "))
        size = int(input("Enter the value of block size: "))
    except ValueError:
        print("Error: n must be an integer.")
        return
    csv_filename = f"cayley_dickson_table_n{n}.csv"
    html_filename = f"cayley_dickson_table_n{n}_exact_order_signs_block_{size}.html"
    block_dimension = 2 ** size
    generate_colored_html_table(csv_filename, html_filename, n, block_dimension)

if __name__ == "__main__":
    main()


# Zero pairs divdor finder

In [ ]:
import re
from functools import lru_cache
from typing import List, Tuple, Dict

# ————————————————————————————————————————————————————————————————
# Core Cayley–Dickson machinery (unchanged)
# ————————————————————————————————————————————————————————————————

def quaternion_sign(i, j):
    """Returns the sign for quaternion multiplication e_i * e_j."""
    table = [
        [1, 1, 1, 1],
        [1, -1, 1, -1],
        [1, -1, -1, 1],
        [1, 1, -1, -1]
    ]
    return table[i][j]

@lru_cache(maxsize=None)
def cayley_dickson_table(n):
    """Generates a multiplication table for the Cayley-Dickson algebra of dimension 2^n."""
    if n == 2:  # Base case: Quaternion table
        dim = 4
        return [[(quaternion_sign(i, j), i ^ j) for j in range(dim)] for i in range(dim)]

    prev_table = cayley_dickson_table(n - 1)
    prev_dim = 1 << (n - 1)
    dim = 1 << n
    table = [[None] * dim for _ in range(dim)]

    for i in range(prev_dim):
        for j in range(prev_dim):
            base_val = prev_table[i][j]
            table[i][j] = base_val
            new_val = (-base_val[0], base_val[1] + prev_dim)
            table[i][j + prev_dim] = new_val
            table[i + prev_dim][j] = new_val
            table[i + prev_dim][j + prev_dim] = new_val

    for i in range(dim):
        local_row = i % prev_dim
        top = (i < prev_dim)
        for j in range(dim):
            local_col = j % prev_dim
            left = (j < prev_dim)
            s, _ = table[i][j]
            if i == 0 or j == 0:
                s = 1
            if i == j:
                s = 1 if i == 0 else -1
            in_a = top and left
            in_b = top and not left
            in_c = not top and left
            in_d = not top and not left
            if local_row == local_col:
                if in_a or in_b:
                    if n > 2 and local_row < 4:
                        s = quaternion_sign(local_row, local_col)
                    elif local_row == 0:
                        s = 1
                    else:
                        s = -1
                elif in_c:
                    s = 1
                elif in_d:
                    s = -1
            if local_col == 0:
                if in_a or in_b:
                    s = 1
                elif in_c or in_d:
                    s = -1 if local_row == 0 else 1
            if local_row == 0:
                if in_a or in_b or (in_c and local_col == 0):
                    s = 1
                elif in_d:
                    s = -1
            if in_d:
                if local_row == 0 and local_col > 0:
                    s = 1
                if local_col == 0 and local_row > 0:
                    s = -1
            table[i][j] = (s, i ^ j)

    return table

# ————————————————————————————————————————————————————————————————
# Parsing & formatting (unchanged)
# ————————————————————————————————————————————————————————————————

_TOKEN_RE = re.compile(r'[+-][^+-]+')
@lru_cache(maxsize=None)
def parse_element(s: str, dim: int):
    # unchanged
    s = s.replace(" ", "")
    if not s:
        raise ValueError("Empty input")
    if s[0] not in "+-":
        s = "+" + s
    coeffs = [0.0] * dim
    for token in _TOKEN_RE.findall(s):
        if "e" in token:
            sign_part, idx_part = token.split("e", 1)
            coeff = 1.0 if sign_part == "+" else -1.0 if sign_part == "-" else float(sign_part)
            idx = int(idx_part)
        else:
            coeff = float(token)
            idx = 0
        if not (0 <= idx < dim):
            raise ValueError(f"Index {idx} out of range")
        coeffs[idx] += coeff
    return coeffs


def format_element(coeffs):
    # unchanged
    terms = []
    for i, c in enumerate(coeffs):
        if c == 0:
            continue
        if i == 0:
            term = f"{c}"
        else:
            if abs(c) == 1:
                term = f"{'+' if c>0 else '-'}e{i}"
            else:
                term = f"{'+' if c>0 else ''}{c}e{i}"
        terms.append(term.replace("+-", "-").replace("++", "+"))
    return "".join(terms) if terms else "0"

# ————————————————————————————————————————————————————————————————
# Equality / opposition (unchanged)
# ————————————————————————————————————————————————————————————————

def are_equal_elements(a, b):
    return a[1] == b[1] and a[0] == b[0]

def are_opposite_elements(a, b):
    return a[1] == b[1] and a[0] == -b[0]

# ————————————————————————————————————————————————————————————————
# Multiplication (unchanged)
# ————————————————————————————————————————————————————————————————

def multiply_elements(a_coeffs, b_coeffs, n):
    table = cayley_dickson_table(n)
    dim = 1 << n
    result = [0.0] * dim
    for i, ai in enumerate(a_coeffs):
        if ai == 0:
            continue
        for j, bj in enumerate(b_coeffs):
            if bj == 0:
                continue
            s, idx = table[i][j]
            result[idx] += ai * bj * s
    return result

# ————————————————————————————————————————————————————————————————
# Normalization & zero‑divisor search (unchanged)
# ————————————————————————————————————————————————————————————————
@lru_cache(maxsize=None)
def normalize_factor(expr: str, dim: int) -> str:
    return format_element(parse_element(expr, dim))

def find_zero_divisor_expressions_exploratory_v4(n: int, i: int, j: int, offset_limit: int):
    dim = 1 << n
    zero_divs = set()
    q = get_element(n, i, j)

    for u in range(1, dim):
        for v in range(1, dim):
            if u == i and v == j:
                continue
            t = get_element(n, u, v)
            if t[1] != q[1]:
                continue

            if are_equal_elements(q, t):
                A_raw = f"e{i}{format_element(parse_element(f'+1e{u}', dim))}"
                B_raw = f"e{j}-{format_element(parse_element(f'-1e{v}', dim))}"
            elif are_opposite_elements(q, t):
                A_raw = f"e{i}{format_element(parse_element(f'+1e{u}', dim))}"
                B_raw = f"e{j}{format_element(parse_element(f'+1e{v}', dim))}"
            else:
                continue

            A = normalize_factor(A_raw, dim)
            B = normalize_factor(B_raw, dim)
            prod = multiply_elements(parse_element(A, dim), parse_element(B, dim), n)
            if format_element(prod) == "0":
                zero_divs.add(f"({A}) * ({B}) = 0")
            continue

    return zero_divs

# ————————————————————————————————————————————————————————————————
# Helper: canonicalize full expression to remove swapped duplicates
# ————————————————————————————————————————————————————————————————

def canonicalize(expr: str) -> str:
    # expr format: "(A) * (B) = 0"
    factors, rest = expr.split(" = ")
    left, right = factors.split(" * ")
    A = left.strip()[1:-1]
    B = right.strip()[1:-1]
    if A > B:
        A, B = B, A
    return f"({A}) * ({B}) = 0"
def get_element(n, i, j):
    """Retrieves the element at row i and column j of the Cayley-Dickson multiplication table."""
    table = cayley_dickson_table(n)
    sign, index = table[i][j]
    return sign, index

# ————————————————————————————————————————————————————————————————
# Driver: modified to dedupe swapped pairs only here
# ————————————————————————————————————————————————————————————————


def normalize_factor_vars(factor: str) -> Tuple[str, ...]:
    """
    Given a factor like "(+e7-e11)", return a sorted tuple of
    the variable names alone, e.g. ("e11","e7").
    """
    vars_ = re.findall(r'e\d+', factor)
    return tuple(sorted(vars_))

def remove_duplicate_equations(equations: List[str]) -> List[str]:
    """
    Remove duplicates of the form "(...)*(...)=0" if they have
    the same two variable‐sets in their factors (ignoring signs).
    """
    seen_keys = set()
    uniques: List[str] = []

    for eq in equations:
        parts = eq.split('=', 1)
        if len(parts) != 2 or parts[1].strip() != '0':
            # Keep anything that isn't "... = 0"
            uniques.append(eq)
            continue

        lhs = parts[0]
        # Split on '*' and strip
        factors = [f.strip() for f in lhs.split('*')]

        # Normalize each factor to its sorted tuple of variable‑names
        norm = [normalize_factor_vars(f) for f in factors]

        # Sort the two factor‐tuples so that A*B ≡ B*A
        key = tuple(sorted(norm))

        # If we've never seen this combination, keep it
        if key not in seen_keys:
            seen_keys.add(key)
            uniques.append(eq)

    return uniques
def main():
    print("Cayley–Dickson Exploratory Zero Divisor Finder (V4)")
    try:
        n = int(input("Enter n (dimension level, n > 3): "))
        if n <= 3:
            print("n must be > 3.")
            return
        dim = 1 << n
        final = set()
        zero_pair_count = 0
    except ValueError:
        print("Invalid integer.")
        return
    try:
        option = int(input("Enter option 1. one element 2. all table: "))
        if option==1:
            i = int(input("Enter option i: "))
            j = int(input("Enter option j: "))
            if i < 0 or i >= dim or j < 0 or j >= dim:
                print(f"Indices i and j are out of range for dimension {dim}")
                return
            block = find_zero_divisor_expressions_exploratory_v4(n, i, j, dim)
            # raw count
            zero_pair_count += len(block)
            # add canonicalized expressions
            for expr in block:
                final.add(canonicalize(expr))
            final=sorted(final)
            final=remove_duplicate_equations(final)
            print(f"\n in the dimension {n} Total distinct zero‑divisor expressions: {len(final)}  (raw count: {zero_pair_count})")
            for expr in sorted(final):
                print(expr)
    except ValueError:
        print("Invalid integer.")
        return
    if option==2:
        for i in range(1, dim):
            for j in range(i, dim):
                if i == j: #or (j - i) >= (dim // 2 + 1):
                    continue
                block = find_zero_divisor_expressions_exploratory_v4(n, i, j, dim)
                # raw count
                zero_pair_count += len(block)
                # add canonicalized expressions
                for expr in block:
                    final.add(canonicalize(expr))
        final=sorted(final)
        final=remove_duplicate_equations(final)
        print(f"\n in the dimension {n} Total distinct zero‑divisor expressions: {len(final)}  (raw count: {zero_pair_count})")

        for expr in sorted(final):
            print(expr)

        print(f"\n in the dimension {n}  Total distinct zero‑divisor expressions: {len(final)}  (raw count: {zero_pair_count})")
if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
(+e75+e100) * (+e83-e124) = 0
(+e75+e100) * (+e84-e123) = 0
(+e75+e100) * (+e87+e120) = 0
(+e75+e100) * (+e88-e119) = 0
(+e75+e100) * (+e91+e116) = 0
(+e75+e100) * (+e92+e115) = 0
(+e75+e100) * (+e95-e112) = 0
(+e75+e101) * (+e77+e99) = 0
(+e75+e101) * (+e8+e38) = 0
(+e75+e101) * (+e80-e126) = 0
(+e75+e101) * (+e83-e125) = 0
(+e75+e101) * (+e85-e123) = 0
(+e75+e101) * (+e86-e120) = 0
(+e75+e101) * (+e88+e118) = 0
(+e75+e101) * (+e91+e117) = 0
(+e75+e101) * (+e93+e115) = 0
(+e75+e101) * (+e94+e112) = 0
(+e75+e102) * (+e78+e99) = 0
(+e75+e102) * (+e80+e125) = 0
(+e75+e102) * (+e83-e126) = 0
(+e75+e102) * (+e85+e120) = 0
(+e75+e102) * (+e86-e123) = 0
(+e75+e102) * (+e88-e117) = 0
(+e75+e102) * (+e91+e118) = 0
(+e75+e102) * (+e93-e112) = 0
(+e75+e102) * (+e94+e115) = 0
(+e75+e103) * (+e79+e99) = 0
(+e75+e103) * (+e8+e36) = 0
(+e75+e103) * (+e80-e124) = 0
(+e75+e103) * (+e83-e127) = 0
(+e75+e103) * (+e84-e120) = 0
(+e75+e103) * (+e87-e123) 

## zero pair counter

In [ ]:
def calculate_zero_divisor_pairs(x):
    """Calculates the number of zero divisor pairs for dimension 2^x using the formula."""
    total_sum = 0.0
    if x <= 3:  # Formula is for x >= 4, and zero divisors are 0 for x <= 3
        return 0
    """
    Old and wrong zero pair divisors counting
      for n in range(x):
        #term = (4**j) * ((2**(x - j) / 8) - 1)  * 2 * ((2**(x - j) / 2) - 1) * ((2**(x - j) / 2) - 2)
    """
    total_sum=((2**x - 2) * (2**x - 4) * (2**x - 8)) / 16
    return int(total_sum)  # Return as integer for number of pairs

print("Zero Divisor Pair Counts for Cayley-Dickson Algebras:")
print("--------------------------------------------------")

results = {}
for x in range(1, 80):  # Dimensions from reals to 80th dimension
    dimension = 1 << x
    pair_count = calculate_zero_divisor_pairs(x) # Using the updated sum function (which is actually the correct formula)
    results[dimension] = pair_count
    cubic_form=2**(3*x-4)
    print(f"Dimension 2^{x} = {dimension}: zero pair count from formula {pair_count} zero divisor pairs {cubic_form} difference {pair_count-cubic_form}")

print("\nTable of Zero Divisor Pair Counts:")
print("{:<15} | {:<25}".format("Dimension", "Zero Divisor Pairs"))
print("--------------------|---------------------------")
for dimension, pair_count in results.items():
    print("{:<15} | {:<25}".format(dimension, pair_count))

Zero Divisor Pair Counts for Cayley-Dickson Algebras:
--------------------------------------------------
Dimension 2^1 = 2: zero pair count from formula 0 zero divisor pairs 0.5 difference -0.5
Dimension 2^2 = 4: zero pair count from formula 0 zero divisor pairs 4 difference -4
Dimension 2^3 = 8: zero pair count from formula 0 zero divisor pairs 32 difference -32
Dimension 2^4 = 16: zero pair count from formula 84 zero divisor pairs 256 difference -172
Dimension 2^5 = 32: zero pair count from formula 1260 zero divisor pairs 2048 difference -788
Dimension 2^6 = 64: zero pair count from formula 13020 zero divisor pairs 16384 difference -3364
Dimension 2^7 = 128: zero pair count from formula 117180 zero divisor pairs 131072 difference -13892
Dimension 2^8 = 256: zero pair count from formula 992124 zero divisor pairs 1048576 difference -56452
Dimension 2^9 = 512: zero pair count from formula 8161020 zero divisor pairs 8388608 difference -227588
Dimension 2^10 = 1024: zero pair count from f